In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

**...**

problemas:

- ~~extrair demora muito tempo, erro dá cabo de tudo:~~

- - ~~funcao por coluna: inp: [col1, col2]: df03=02.copy, replace col, save, replace col, save...se der erro, só importa a ultima col~~

- - se fizer de vez keywords/sentiment/... assim poupa-se tempo do 1min dos requests

- noticias sobre algo, e contém a palavra Galp, como sei oq interessa?

- - https://arquivo.pt/textextracted?m=http%3A%2F%2Fwww.dn.pt%2Fneg%2Fpub%2F10s19a.htm%2F20010916233215

- 1. se resumir a noticia e n tiver a palavra galp, ignoro? - ver qnt demora

- 2. selecionar frase que tem a palavra +-1 frase, e pronto ? - n é má ideia se n se perder muita informcao

- 3. noticia tem de ter pelo menos 3? mencoes da empresa/aliases - ver quanto reduz

- 4. **logistical regression: machine learning: features: qty aliases, proporcao alfanumerico, qty palavras, ...**

- filtrar noticias muito parecidas

- 1. **fazer NER/sentiment, dois iguais, ver se é 80% igual?**

- 2. selecionar 4 frases aleatórias ou contar palavras, e se for igual pronto?

- 3. "bbiggest substring" - se >80%, é igual ?

- que modelo usar para Sentiment Analysis ?

- converter noticias.txt into noticias.csv - para dps associar o site do arquivo ao nome da fonte de noticias


tentar NER + Sentiment Analysis

# CRIAR NOVO CABECALHO

# REMOVER 80% IGUAL OU MERGE DIFFERENCES

# ....

In [5]:

import re

def extract_information(text, keyword):
    # Compile a regex pattern to find whole sentences containing the keyword
    pattern = re.compile(r'([^.]*?\b' + re.escape(keyword) + r'\b[^.]*\.)', re.IGNORECASE)
    
    # Find all matches for the keyword in the text
    matches = pattern.findall(text)
    
    # If matches are found, return them; otherwise, return an informative message
    if matches:
        return ' '.join(matches)
    else:
        return f"No information found about {keyword}."

# Example usage with your provided text

extract_information(text, "Galp")

' Alguns exemplos: Presidência do Conselho de Ministros, ministérios da Economia e das Finanças, Galp Energia, TAP, grupo EDP e RTP.  Refiram-se alguns exemplos sintomáticos: Presidência do Conselho de Ministros, ministérios da Economia e das Finanças, Galp Energia, TAP, grupo EDP e RTP.'

In [5]:
galp = pd.read_parquet("data02.parquet")["api.2003"].loc["Mota-Engil"]
galp

array([{'ExtractedText': 'RTP Empresas portuguesas sem condições para participar na reconstrução do Iraque As condições exigidas pela empresa norte-americana responsável pela reconstrução do Iraque impedem as portuguesas de participar, segundo as maiores construtoras nacionais. A U.S. Agency for International Development (USAID) adjudicou as obras de reconstrução no Iraque à norte-americana Bechtel por 581 milhões de euros (680 milhões de dólares), que irá sub-empreitar 90 por cento das obras. No entanto, segundo explicou o responsável pelos negócios internacionais da Somague, José Paulo Assunção, "as sub- empreitadas da Bechtel não irão além de um milhão de dólares (855 milhões de euros), ou seja, serão projectos cirúrgicos, pouco interessantes". "Ninguém vai para um país distante e problemático como o Iraque por contratos daquela dimensão, para reconstruir uma porta ou uma janela", ironizou, por sua vez, o administrador da Mota- Engil, Sérgio Arnaldo Figueiredo. As exigências de segu

In [23]:
import spacy
from transformers import pipeline

# Carregar o modelo de NLP do spaCy
nlp = spacy.load("pt_core_news_sm")

# Inicializar o pipeline de sumarização
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="pt")


# Função para extrair partes do texto relacionadas à empresa e gerar um resumo
def extrair_resumo(texto, nome_empresa):
    # Passo 1: Identificar entidades mencionadas no texto
    doc = nlp(texto)
    # Passo 2: Filtrar sentenças que mencionam a empresa
    sentencas_relevantes = [sent.text for sent in doc.sents if nome_empresa.lower() in sent.text.lower()]
    # ISTO FAZ COM Q SE IGNORE TUDO  O Q N DIZ GALP !!!!
    
    # Passo 3: Se encontrarmos sentenças relacionadas à empresa, concatenamos e fazemos um resumo
    if sentencas_relevantes:
        texto_relevante = " ".join(sentencas_relevantes)
        # Gerar um resumo a partir do texto relevante
        resumo = summarizer(texto_relevante, max_length=len(texto_relevante)*0.5, min_length=30, do_sample=False) #max_length=100, min_length=30
        return resumo[0]['summary_text']
    
    return "Nenhuma informação relevante sobre a empresa foi encontrada."

# Exemplo de uso
#texto_exemplo = """
#A empresa X tem crescido no mercado de tecnologia. Ela recentemente fez uma parceria com a Y corp.
#No entanto, alguns críticos dizem que o mercado global pode impactar negativamente. Em 2023, a empresa X lançou novos produtos no setor de IA.
#"""

df = pd.read_parquet("data01.parquet")["api.2000"].loc["Galp Energia"]
texto_exemplo = fetch_with_retry(df[0]["linkToExtractedText"])


nome_empresa = "Galp"
resumo = extrair_resumo(texto_exemplo, nome_empresa)
print(resumo)


DN-Suplementos Negócios 10 de Setembro de 2001 Agências nacionais "esquecidas" pelo Estado Prevenção Rodoviária não convidou empresas portuguesas para o seu concurso publicitário Cátia Almeida DN-Natacha Cardoso Em pé de igualdade. Agências nacionais são tão boas como as multinacionais, afirma António Ferreira A Prevenção Rodoviária Portuguesa (PPR) abriu um concurso com o objectivo de seleccionar uma agência para desenvolver a sua próxima campanha de publicidade. Só que, para participar no concurso, apenas foram convidadas empresas que fazem parte de cadeias multinacionais (oito), ficando de lado as agências portuguesas. Para António Ferreira, director de serviço a clientes da MKT (agência do grupo Central de Comunicação), é inadmissível que, "num concurso com dinheiro dos contribuintes, as empresas nacionais não possam sequer apresentar uma proposta. Este caso, que não é único, ilustra uma atitude de preconceito para com aquilo que é nacional". "Há excelentes empresas portuguesas que